In [1]:
import pandas as pd
import numpy as np
import yaml
import json
import os
import glob
import re

In [2]:
ASGN_NAME = 'assignment_2739086_export'

meta = yaml.safe_load(
    open(f'{ASGN_NAME}/submission_metadata.yml', 'r')
    )

projs = pd.read_csv('Public-Facing Block Assignments – DSC 180AB, 2022-23 - Sheet1.csv')
projs = projs.iloc[1:].reset_index(drop=True)
projs.head()

def load_artifacts(sub):
    artifacts = json.load(open(f'{ASGN_NAME}/{sub}/artifacts.json', 'r'))
    code = artifacts['project-repository']
    code = code.replace('.git', '')
    website = artifacts['project-website-url']
    return code, website

def load_title_abstract(sub):
    f = open(f'{ASGN_NAME}/{sub}/title-abstract.txt', 'r').read()
    try:
        title = re.findall(r'Title:\n?(.+)', f)[0]
    except:
        return(sub)
    title = title.strip().replace('<', '').replace('>', '')
    abstract = re.findall(r'Abstract:\n?(.+)', f)[0]
    return title, abstract

def copy_poster(sub, group):
    os.system(f'cp {ASGN_NAME}/{sub}/report.pdf ../reports/{group}.pdf')
    
out_df = pd.DataFrame(columns=['Group #', 'Code', 'Website', 'Title', 'Abstract'])

for sub in meta:
    try:
        name = meta[sub][':submitters'][0][':name']
    except Exception as e:
        print(sub)
    row = projs[projs['Names'].str.contains(name)]
    if len(row) != 1:
        print(f'name found {len(row)} times in names')
    else:
        row = row.iloc[0]
        group = row['Group #']
        try:
            code, website = load_artifacts(sub)
            title, abstract = load_title_abstract(sub)
            copy_poster(sub, group)
            s_dict = {'Group #': group, 
                      'Code': code, 
                      'Website': website, 
                      'Title': title,
                      'Abstract': abstract}
            out_df = pd.concat([out_df, pd.DataFrame([s_dict])])
        except Exception as e:
            print(e)
    
out_df = out_df.groupby('Group #').last().reset_index()
out_df